In [1]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

C:\Users\Naren\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [141]:
input_path = r"C:\Users\Naren\Dropbox\electoral_rolls_matching\up_ocr\include_left\66_3.xls"
output_directory = r"C:\Users\Naren\Dropbox\electoral_rolls_matching\up_ocr\include_left\processed"

filename = input_path.rsplit(os.sep)[-1]
output_path = os.path.join(output_directory, filename)

In [142]:
ocr = pd.read_excel(input_path)

### create a new column in ocr spreadsheet called "processed" which is blank if ocr failed on left side of image, says to ignore the village on the left and use mukhya village if the two match closely, and is the same as the left side name otherwise (stripped of non-letter symbols)

In [143]:
column = []
for i in ocr.index:
    if pd.isna(ocr["Name_Cleaned_Hin"][i]):
        if pd.isna(ocr["mukhya_village"][i]):
            column.append("failure")
        else:
            column.append("")
        continue
    stripped = ocr["Name_Cleaned_Hin"][i].rstrip(" +-1234567890१२३४५६७८९|\"")
    if stripped.endswith("आंशिक"):
        stripped = stripped.rsplit("आंशिक", maxsplit = 1)[0]
    if stripped.endswith("अंश"):
        stripped = stripped.rsplit("अंश", maxsplit = 1)[0]
    stripped = stripped.rstrip(" +-1234567890१२३४५६७८९|\"")
    if pd.isna(ocr["mukhya_village"][i]):
        column.append(stripped)
        continue
    t_left = transliterate(stripped, sanscript.DEVANAGARI, sanscript.OPTITRANS)
    t_right = transliterate(ocr["mukhya_village"][i], sanscript.DEVANAGARI, sanscript.OPTITRANS)
    if fuzz.ratio(t_left, t_right) > 80:
        column.append("(ignore)")
    else:
        column.append(stripped)
ocr["processed"] = column

## go through the dataframe thus generated and create a new row with mukhya gaon for every image where the mukhya gaon is not found to exist in the left-side list

In [144]:
k = ocr["ps_id"][0]
foundMain = False
rows = []
for i in ocr.index:
    if k != ocr["ps_id"][i]: #ps_id change
        if not foundMain and not pd.isna(mukhya_gaon): 
            new_row = pd.DataFrame({"ps_id":k, 
                                    "Name_Cleaned_Hin":'', 
                                    "mukhya_village":mukhya_gaon, 
                                    "block":block, 
                                    "tehsil":tehsil, 
                                    "processed":'(ignore)'}, index = [i])
            rows.append(new_row)
        foundMain = False
    k = ocr["ps_id"][i]
    mukhya_gaon = ocr["mukhya_village"][i]
    block = ocr["block"][i]
    tehsil = ocr["tehsil"][i]
        
    if ocr["processed"][i] == "(ignore)" or pd.isna(ocr["Name_Cleaned_Hin"][i]):
        foundMain = True
last = len(ocr) - 1
if not foundMain and not pd.isna(ocr["mukhya_village"][last]):
    new_row = pd.DataFrame({"ps_id":ocr["ps_id"][last], 
                            "Name_Cleaned_Hin":'', 
                            "mukhya_village":ocr["mukhya_village"][last], 
                            "block":ocr["block"][last], 
                            "tehsil":ocr["tehsil"][last], 
                            "processed":'(ignore)'}, index = [last])
    rows.append(new_row)

## insert the rows into a dataframe

In [145]:
offset = 0
new_df = ocr
for row in rows:
    new_df = pd.concat([new_df.iloc[:row.index[0] + offset], row, new_df[row.index[0] + offset:]]).reset_index(drop = True)
    offset += 1

## save to excel

In [146]:
new_df.to_excel(output_path, index = False)